<a href="https://colab.research.google.com/github/danielmeans/DeepLearning-291K-/blob/master/PixelRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tf-nightly-gpu-2.0-preview
import tensorflow as tf


In [2]:
from keras.datasets import cifar10
(x_train, y_train_), (x_test, y_test_) = cifar10.load_data()
x_train_n = x_train.astype('float32') / 255
x_test_n = x_test.astype('float32') / 255

from keras.utils import to_categorical
y_train = to_categorical(y_train_)
y_test = to_categorical(y_test_)

Using TensorFlow backend.


In [0]:
import numpy as np
height, width, channels, flat_dim = 32, 32, 3, 3072

#For each pixel in Each image, construct vectors of neightboring pixels
# Flatten each channel into a 1024 length 1-dimensional vector
def get_pixel_neighbor_vector(x_train):
  image = x_train[0]
  image_r = np.reshape(image, (3, -1))
  #Number of preceding pixels to use when predicting current pixel
  num_neighbors = 19
  #tensor to store neighboring pixels of each pixel in an image
  neighbor_tensor = []
  for channel in range(channels):
    per_channel_tensor = []
    for index in range(flat_dim):
      #determine how much padding to add to the neighboring pixels, for the first 19 pixels
      start = max(index - num_neighbors, 0)
      padding_length = num_neighbors - (index - start)
      padding = np.array([0] * padding_length)
      #append place holders for red, green channels computed for these pixels
      if channel == 1:
        red_green = np.concatenate((np.array(image_r[0][index: index + 1 ]), np.array([0])), axis=0)
      elif channel == 2:
        red_green = np.concatenate((np.array(image_r[0][index: index + 1]), np.array(image_r[1][index: index + 1])), axis=0)
      else:
        red_green = np.array([0] * 2)
      neighbors = np.concatenate((padding, image_r[channel][start:index], red_green), axis=0)
      per_channel_tensor.append(neighbors)
    neighbor_tensor.append(per_channel_tensor)
  return neighbor_tensor
def get_pixel_neighbor_vector_flat(image):
  #Flatten image (32 * 32 * 3) --> 3072
  image_r = np.reshape(image,  (-1))
  #Number of preceding pixels to use when predicting current pixel
  num_neighbors = 21
  #tensor to store neighboring pixels of each pixel in an image
  neighbor_tensor = []
  for index in range(flat_dim):
    #determine how much padding to add to the neighboring pixels, for the first 19 pixels
    start = max(index - num_neighbors, 0)
    padding_length = num_neighbors - (index - start)
    padding = np.array([0] * padding_length)
    neighbors = np.concatenate((padding, image_r[start:index]), axis=0)
    neighbor_tensor.append(neighbors)
  return np.array(neighbor_tensor).astype('float32')

def get_pixel_neighbor_vector_2(image):
  #Number of preceding pixels to use when predicting current pixel
  num_neighbors = 6
  #tensor to store neighboring pixels of each pixel in an image
  neighbor_tensor = []
  for row in range(len(image)):
    for col in range(len(image[0])):
      #determine how much padding to add to the neighboring pixels, for the first 19 pixels
      start = max(col - num_neighbors, 0)
      padding_length = num_neighbors - (col - start)
      padding = np.array([0] * padding_length)
      for pixel in range(start, col):
        for channel in range(len(pixel)):
          
          #append place holders for red, green channels computed for these pixels
          if channel_idx == 1:
            red_green = np.concatenate((np.array(image[row][col][channel_idx - 1]), np.array([0])), axis=0)
          elif channel_idx == 2:
            red_green = np.concatenate((np.array(image[row][col][channel_idx -2 ]), np.array(image[1][col: col + 1][channel_idx -2])), axis=0)
          else:
            red_green = np.array([0] * 2)
          
        
      neighbors = np.concatenate((padding, image[channel_idx][start:col], red_green), axis=0)
  return neighbor_tensor



In [0]:
import numpy as np
image_r = np.reshape(x_train[0], (32, 96))
first_row = image_r[1]

def get_one_hot_vectors(image):
  while True:
    images=[]
    for row in range(height):
      cur_row = image[row]
      one_hot_row = []
      for pix in cur_row:
        pixel = []
        for pix_channel in pix:
          one_hot_pix_channel = [0] * 256
          one_hot_pix_channel[pix_channel] = 1
          pixel.append(one_hot_pix_channel)
        one_hot_row.append(pixel)
      images.append(one_hot_row)
    yield np.array(images).astype('int32')
    


In [0]:
# pixel by pixel rnn
from numpy import array
# define input sequence
for i in range(len(x_train()): 
  neighbors = get_pixel_neighbor_vector_flat(x_train_n[i])
  one_hot_image_gen = get_one_hot_vectors(x_train[i])
  one_hot_image = next(one_hot_image_gen)
  for row in range(len(one_hot_image)):
    for col in range(len(one_hot_image[0])):
      for channel in range(len(one_hot_image[0][0])):
        neighbor_idx = row * 32 + col * 3 + channel
        seq_in = neighbors[neighbor_idx]
        # reshape input into [samples, timesteps, features]
        n_in = len(seq_in)
        seq_in = seq_in.reshape((1, n_in, 1))
        # prepare output sequence
        seq_out = one_hot_image[row][col][channel]
        n_out = len(seq_out)
        seq_out = seq_out.reshape((1, n_out))
        #define model
        model = tf.keras.models.Sequential()
        tf.keras.layers.Dense(units=512, activation=tf.nn.relu, use_bias=True, input_shape=(n_in, 1))
        model.add(tf.keras.layers.LSTM(512, activation='relu', use_bias=True))
        model.add(tf.keras.layers.RepeatVector(n_out))
        model.add(tf.keras.layers.LSTM(512, activation='relu',  use_bias=True))
        #model.add(tf.keras.layers.Dense(1))
        model.add(tf.keras.layers.Dense(256, activation='softmax'))
        with tf.device('/device:GPU:0'):

          model.compile(optimizer='adam', loss=tf.nn.softmax_cross_entropy_with_logits)

          # fit model
          model.fit(seq_in, seq_out, epochs=100, verbose=1)
          # demonstrate prediction
          yhat = model.predict(seq_in, verbose=0)
          print(yhat)
          print(seq_out)




In [65]:
###Row LSTM
#define model
for image in x_train:
  for row in image:
    seq_in = row
    # reshape input into [samples, timesteps, features]
    n_in = len(seq_in)
    seq_in = seq_in.reshape((1, n_in, 1))
    # prepare output sequence
    seq_out = seq_in[:,:,:]
    with tf.device('/device:GPU:0'):
      model =  tf.keras.models.Sequential()
      model.add(tf.keras.layers.LSTM(100, activation='relu', input_shape=(n_in,1)))
      model.add(tf.keras.layers.RepeatVector(n_in))
      model.add(tf.keras.layers.LSTM(100, activation='relu', return_sequences=True))
      model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1)))
      #compile model
      model.compile(optimizer='adam', loss=tf.nn.softmax_cross_entropy_with_logits)

      # fit model
      model.fit(seq_in, seq_out, epochs=100, verbose=1)
      # demonstrate prediction
      yhat = model.predict(seq_in, verbose=0)




Train on 1 samples
Epoch 1/100
1/1 [==============================] - 2s 2s/sample - loss: 5.5452
Epoch 2/100
1/1 [==============================] - 0s 313ms/sample - loss: 5.5452
Epoch 3/100
1/1 [==============================] - 0s 296ms/sample - loss: 5.5452
Epoch 4/100
1/1 [==============================] - 0s 294ms/sample - loss: 5.5452
Epoch 5/100
1/1 [==============================] - 0s 287ms/sample - loss: 5.5451
Epoch 6/100
1/1 [==============================] - 0s 299ms/sample - loss: 5.5451
Epoch 7/100
1/1 [==============================] - 0s 301ms/sample - loss: 5.5451
Epoch 8/100
1/1 [==============================] - 0s 295ms/sample - loss: 5.5451
Epoch 9/100
1/1 [==============================] - 0s 317ms/sample - loss: 5.5451
Epoch 10/100
1/1 [==============================] - 0s 296ms/sample - loss: 5.5451
Epoch 11/100
1/1 [==============================] - 0s 284ms/sample - loss: 5.5451
Epoch 12/100
1/1 [==============================] - 0s 307ms/sample - loss: 5.54